In [1]:
from dotenv import load_dotenv

_ = load_dotenv(dotenv_path='../.env/.env')

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
type(AgentState)

typing._TypedDictMeta

In [10]:
test = AgentState({'messages':['test']})

In [11]:
test

{'messages': ['test']}

In [12]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(':memory:')

In [14]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [20]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [21]:
messages = [HumanMessage(content='What is the weather in SF?')]

In [22]:
thread = {'configurable':{'thread_id':'1'}}

In [23]:
for event in abot.graph.stream({'messages': messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hRVmDlGG9HVDSCgUcfbKSSSf', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 164, 'total_tokens': 186}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_486730399b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-53adc345-844d-47a5-8c87-9fad9f49324e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_hRVmDlGG9HVDSCgUcfbKSSSf'}], usage_metadata={'input_tokens': 164, 'output_tokens': 22, 'total_tokens': 186})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_hRVmDlGG9HVDSCgUcfbKSSSf'}
Back to the model!
[ToolMessage(content="[{'url': 'https://world-weather.info/forecast/usa/san_francisco/june-2024/

In [24]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_er9ldHR4GALe1pXluxXTwDp8', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 944, 'total_tokens': 966}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_486730399b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fdf0c8f3-1234-4f93-bd4c-e5c647a62c95-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_er9ldHR4GALe1pXluxXTwDp8'}], usage_metadata={'input_tokens': 944, 'output_tokens': 22, 'total_tokens': 966})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_er9ldHR4GALe1pXluxXTwDp8'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content

In [25]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='San Francisco is currently warmer with a temperature of 72°F compared to Los Angeles, which has a temperature of 71.1°F.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 1524, 'total_tokens': 1553}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_486730399b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b783f5e1-ed75-4618-b1ed-8309ca13beae-0', usage_metadata={'input_tokens': 1524, 'output_tokens': 29, 'total_tokens': 1553})]}


In [26]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please specify what items or places you are comparing in terms of warmth?', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 151, 'total_tokens': 168}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_486730399b', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d8ea30b-708f-4b64-88fd-2382142347d2-0', usage_metadata={'input_tokens': 151, 'output_tokens': 17, 'total_tokens': 168})]}


In [27]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

ModuleNotFoundError: No module named 'aiosqlite'